In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [10]:
!pip install -q langgraph-checkpoint-sqlite


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Persistent In-memory Database

In [11]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect(":memory:", check_same_thread=False)
memory = SqliteSaver(conn)

# memory = SqliteSaver.from_conn_string(":memory:")

In [12]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [13]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [14]:
messages = [HumanMessage(content="What is the weather of sf?")]

In [15]:
thread = {"thread_id": "1"}

In [16]:
for event in abot.graph.stream({"messages": messages}, config=thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TeqzWX3aHAhs3MgfCZXM3H0J', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 151, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0d4eb8a50b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-12106431-b642-4b8d-87dd-6afc20c58903-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_TeqzWX3aHAhs3MgfCZXM3H0J', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 23, 'total_tokens': 174, 'input_token_details'

In [17]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5qTntUNy3lSsVgZ0uddDyRcc', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 1070, 'total_tokens': 1093, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1024}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0d4eb8a50b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b0638ca3-bf6a-4723-b187-5f69e0c8da04-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_5qTntUNy3lSsVgZ0uddDyRcc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1070, 'output_tokens': 23, 'total_tokens': 1093, 'inpu

In [18]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Both San Francisco and Los Angeles have the same current temperature of 13.3°C (55.9°F). Therefore, neither city is warmer than the other at the moment.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 1987, 'total_tokens': 2025, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1920}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0d4eb8a50b', 'finish_reason': 'stop', 'logprobs': None}, id='run-1fe2e2af-1237-49bc-8ee4-6ea98d04931c-0', usage_metadata={'input_tokens': 1987, 'output_tokens': 38, 'total_tokens': 2025, 'input_token_details': {'audio': 0, 'cache_read': 1920}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [19]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="I need more context to provide an accurate response. Are you comparing the climate of different locations, the temperature of different materials, or something else? Please provide more details, and I'll be happy to help!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 149, 'total_tokens': 192, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0d4eb8a50b', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b1210f3-cddf-4358-9577-6e9b3bf18aa0-0', usage_metadata={'input_tokens': 149, 'output_tokens': 43, 'total_tokens': 192, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


# Streaming tokens

In [31]:
!pip -q install aiosqlite


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
import sqlite3

conn = sqlite3.connect(":memory:")
memory = AsyncSqliteSaver(conn)

In [41]:
# from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver
import asyncio

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [43]:
# messages = [HumanMessage(content="What is the weather in SF?")]
# thread = {"configurable": {"thread_id": "4"}}
# async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
#     kind = event["event"]
#     if kind == "on_chat_model_stream":
#         content = event["data"]["chunk"].content
#         if content:
#             # Empty content in the context of OpenAI means
#             # that the model is asking for a tool to be invoked.
#             # So we only print non-empty content
#             print(content, end="|")

# Sqlite Dabase

In [44]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect("memory_store.db", check_same_thread=False)
memory = SqliteSaver(conn)

In [45]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [46]:
messages = [HumanMessage(content="What is the weather of sf?")]

In [47]:
thread = {"configurable": {"thread_id": "1"}}

In [48]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sHgdJRxtlQ3jY8xFyqjj7Jaf', 'function': {'arguments': '{"query":"San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0d4eb8a50b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b2f30d11-8328-495b-b451-a1cf59b1db75-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_sHgdJRxtlQ3jY8xFyqjj7Jaf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 22, 'total_tokens': 173, 'input_token_details': {'audio'

In [51]:
thread = {"configurable": {"thread_id": "1"}}
previous_state = memory.get(thread)

In [54]:
previous_state["channel_values"]["messages"]

[HumanMessage(content='What is the weather of sf?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sHgdJRxtlQ3jY8xFyqjj7Jaf', 'function': {'arguments': '{"query":"San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_0d4eb8a50b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b2f30d11-8328-495b-b451-a1cf59b1db75-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_sHgdJRxtlQ3jY8xFyqjj7Jaf', 'type': 'tool_call'}], usage_metadat